# Step-by-Step Invoice Extraction with Nanonets OCR LLM
This notebook guides you through running the invoice field extractor over markdown outputs generated by the Nanonets, PaddleOCR, and Tesseract pipelines.

## Prerequisites
- Python 3.10 or later
- Access to the `nanonets/Nanonets-OCR-s` model on Hugging Face
- Markdown invoice outputs located in `nanonet_markdown_output/`, `paddle_markdown_output/`, and `tesseract_markdown_output/`

If you have not installed the required libraries yet, run the next cell.

In [1]:
%pip install torch transformers pillow

Note: you may need to restart the kernel to use updated packages.


## 1. Configure Paths and Define Helpers
Import the required libraries and declare helper functions for prompting the LLM, parsing its output, and iterating over the markdown directories.

In [2]:
import json
import logging
import re
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Iterable, Optional

import torch
from transformers import (
    AutoModelForImageTextToText,
    AutoProcessor,
    AutoTokenizer,
)

try:
    import flash_attn  # noqa: F401

    HAS_FLASH_ATTN = True
except ImportError:
    HAS_FLASH_ATTN = False

DEFAULT_MODEL_PATH = "nanonets/Nanonets-OCR-s"
DEFAULT_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LOGGER = logging.getLogger("invoice_extractor")


@dataclass
class ExtractionResult:
    """Structured fields extracted from an invoice."""

    invoice_number: Optional[str]
    billing_address: Optional[str]
    total_amount: Optional[str]

    @classmethod
    def from_json(cls, payload: Dict[str, Optional[str]]) -> "ExtractionResult":
        return cls(
            invoice_number=payload.get("invoice_number"),
            billing_address=payload.get("billing_address"),
            total_amount=payload.get("total_amount"),
        )

    def to_dict(self) -> Dict[str, Optional[str]]:
        return {
            "invoice_number": self.invoice_number,
            "billing_address": self.billing_address,
            "total_amount": self.total_amount,
        }


def configure_logging(verbose: bool = False) -> None:
    level = logging.DEBUG if verbose else logging.INFO
    logging.basicConfig(level=level, format="%(asctime)s [%(levelname)s] %(message)s")


def load_llm(
    model_path: str = DEFAULT_MODEL_PATH,
    prefer_flash_attn: bool = True,
    device: str = DEFAULT_DEVICE,
):
    LOGGER.info("Loading model '%s'", model_path)
    LOGGER.info("Target device: %s", device)

    attn_kwargs = {}
    if prefer_flash_attn and HAS_FLASH_ATTN:
        LOGGER.info("Using FlashAttention 2 backend.")
        attn_kwargs["attn_implementation"] = "flash_attention_2"
    elif prefer_flash_attn:
        LOGGER.warning(
            "FlashAttention 2 not available; falling back to default attention. "
            "Install flash-attn to enable it."
        )

    model = AutoModelForImageTextToText.from_pretrained(
        model_path,
        torch_dtype="auto",
        device_map=None,
        **attn_kwargs,
    )
    model.to(device)
    model.eval()

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    processor = AutoProcessor.from_pretrained(model_path)
    return model, tokenizer, processor


def build_prompt(markdown_text: str) -> str:
    instructions = (
        """You are an expert financial document analyst. Your task is to extract structured data from invoices.

You will be given OCR text from an invoice inside <document>...</document> tags.

Your responsibilities:

Identify the language of the text.

Understand the content in that language.

Extract ONLY the requested fields using the rules below.

Return a STRICTLY valid minified JSON object.

You MUST follow all rules below. Do not ignore or override any rule.

INPUT FORMAT

You will receive input in this form:

<document> [OCR text from an invoice...] </document>

You MUST follow this SCOPE RULE (CRITICAL):

You are ONLY allowed to extract values that appear literally, verbatim, inside the <document>...</document> block.

You are FORBIDDEN from copying, reusing, paraphrasing, or inferring any value from:

The instructions in this prompt,

The examples in this prompt,

Your own prior knowledge,

Any previous invoices.

If a value is not explicitly present in the <document> text, you MUST return null for that field.

Never guess. Never invent. Never assume formatting.

Never use any company name, address, currency amount, date, ID, tax number, or any other content from the examples in this prompt unless it is ALSO present in the current <document> text.

Only analyze text INSIDE the <document>...</document> tags.
Ignore anything outside those tags.

OUTPUT FORMAT

Your entire response must be ONLY a single minified JSON object with EXACTLY these keys in this exact order:

{
"invoice_number": "...",
"invoice_date": "...",
"billing_address": "...",
"total_amount": "..."
}

Rules:

The JSON must be valid.

Keys must appear in this exact order: invoice_number, invoice_date, billing_address, total_amount.

Strings must be valid JSON strings with proper escaping.

billing_address line breaks must be encoded with "\n".

If a field cannot be confidently found, set it to null (without quotes), e.g. "invoice_number": null.

Do NOT include any commentary, notes, markdown, bullets, reasoning steps, or explanations outside of the JSON.

Do NOT add extra keys.

Do NOT output anything other than the JSON object.

GLOBAL EXTRACTION RULES

Extract ONLY what is required for each field.

Do NOT merge multiple fields together.

Do NOT invent or infer values that are not present.

Do NOT include unrelated details such as tax info, GST numbers, phone numbers, emails, IBAN, VAT IDs, etc., unless explicitly allowed.

If multiple candidates exist for a field, follow the priority rules described for that specific field.

If you are unsure, you MUST answer null. Guessing is never allowed, even if other invoices in the examples had a similar pattern.

If you cannot find a clear value for a field directly in the <document> text, output null for that field. Do not guess, infer, fabricate, or copy anything from the examples in this prompt.

You must follow all extraction rules even if OCR text is noisy, broken across lines, or partially corrupted.

FIELD DEFINITIONS AND RULES

invoice_number

Definition:
The unique invoice identifier that represents this specific invoice/bill.

How to detect:
Look for labels such as (case-insensitive and punctuation-insensitive):

"INV. NO.", "INV NO", "INV NO."

"INVOICE NO", "INVOICE NO.", "INVOICE #"

"INVOICE NUMBER", "INVOICE NUM", "INVOICE ID"

"INVOICE:", "INVOICE"

"Invoice No", "Invoice no:"

"Bill Number", "Bill No", "BILL NO.", "BILL #"

This value is usually found near other header fields like "DATE", "TERMS", "DUE DATE", "DATE OF ISSUE", "PAGE", etc.

Rules:

Return ONLY the value, not the label.
Example: If OCR shows INVOICE NO.: 31T 2552-14B, you must return "31T 2552-14B".

Preserve punctuation, dashes, slashes, spaces, and casing exactly as shown in the OCR.

Do NOT join or combine multiple numbers.

If multiple invoice-like numbers appear:

Prefer the one in the primary invoice header block alongside fields like "Date", "Date of issue", "Terms", etc.

Prefer the one closest to wording like "Invoice", "Invoice no", "Invoice number", etc.

Null handling:

If you cannot confidently identify any invoice identifier directly in the <document> text, set "invoice_number": null.

You MUST NOT copy an invoice number from the examples unless that exact number is present in the current <document>.

invoice_date

Definition:
The official date of the invoice. This is the billing/issue date of the invoice.

How to detect:
Look for labels such as:

"DATE"

"INVOICE DATE"

"BILL DATE"

"ISSUE DATE"

"DATE OF INVOICE"

"TAX INVOICE DATE"

"Date of issue:" (treat this as the invoice date)

Rules:

Return ONLY the date string exactly as it appears in the OCR.

Do NOT reformat. Keep the original style, e.g. "March 8, 1984", "08/03/1984", "2024-11-30", "04/13/2013".

Do NOT include label text like "DATE:" or "Date of issue:" in the value. Only the date itself.

If multiple dates appear (Invoice Date, Due Date, Delivery Date, Period Start/End, etc.):

Prefer "DATE:", "INVOICE DATE:", "Date of issue:", "ISSUE DATE:" over "DUE DATE:" or "DELIVERY DATE:".

Do NOT select range-style periods like "February 1, 1984 to February 29, 1984". That is not the invoice date.

Null handling:

If you cannot confidently locate the invoice/issue/bill date directly in the <document> text, set "invoice_date": null.

NEVER copy a date from the examples. Only use a date if it appears in the current <document>.

billing_address

Definition:
The address of the entity being billed (the customer / buyer / client who is expected to pay).

Important:
This is NOT the seller/supplier address.

How to detect:
Look for blocks introduced by labels like:

"BILL TO", "BILL TO:", "BILLED TO", "BILLED TO:"

"BILLING ADDRESS", "BILLING ADDRESS:"

"Bill To Address"

"INVOICE TO", "INVOICE TO:"

"Customer", "Customer Address", "Customer Address:"

"Client:" or "Client" (treat the block under "Client:" as the billed party if it is clearly the buyer/customer)

"TO:" (only when it clearly refers to the recipient/customer being invoiced, not the sender)

Do NOT confuse with:

"Seller:", "Vendor:", "Supplier:", "From:" → those identify the issuer/seller, NOT the billed customer. You MUST NOT use those for billing_address.

What to capture:

Include:

The billed company's name or recipient name.

The street / unit / address lines that follow.

City, state, postal code lines.

Also include, if present in the SAME billed party block:

Contact person lines introduced with "ATTN:" or "Attention:".

A job title line immediately after that (for example "Director, Research") if it's clearly still part of the contact at that billing address.

You MUST preserve line order from the invoice.

In the output JSON string, represent line breaks as "\n".

How to handle "ATTN:" / "Attention:" lines:

Include ONLY the text AFTER "ATTN:" / "Attention:" as its own line.
Example:
"ATTN: Dr. Harry Minnemeyer"
becomes
"Dr. Harry Minnemeyer"
If the next line is that person's title (for example "Director, Research"), include it as another line.

Stopping conditions:
Stop collecting lines for billing_address when:

You hit another clearly labeled block (for example "SHIP TO", "SELLER", "VENDOR", "FROM", "SUPPLIER", "SUMMARY", "ITEMS", "TERMS", "TOTAL", etc.).

The text changes to unrelated content like "Tax Id:", "IBAN:", "VAT:", phone numbers, emails, etc.

VERY IMPORTANT EXCLUSIONS:

You MUST NOT include "Tax Id", "IBAN", VAT numbers, phone numbers, emails, etc., even if they appear directly after the address block.

You MUST NOT include the seller/supplier/vendor address.

You MUST NOT merge two different addresses.

You MUST NOT invent a company name or address based on prior examples.

If the OCR does not clearly expose a billed-party block ("Client:", "Bill To:", "Customer", etc.), you MUST return null for billing_address. Do NOT guess.

Output formatting for billing_address:

The final value in JSON must be a single string.

Replace actual line breaks with the literal characters "\n".

Example source:

BILL TO:
Lorillard Research Center
Post Office Box 21688
Greensboro, N. C. 27420
ATTN: Dr. Harry Minnemeyer
Director, Research

Expected "billing_address":
"Lorillard Research Center\nPost Office Box 21688\nGreensboro, N. C. 27420\nDr. Harry Minnemeyer\nDirector, Research"

Null handling:

If NO billing/customer/client block is present in the <document> text, set "billing_address": null.

You MUST NOT copy any address from the examples unless that exact address appears in the current <document>.

total_amount

Definition:
The final amount payable for this invoice — the amount the billed party actually owes. This is the amount after tax, not just pre-tax subtotal.

LEGAL SOURCE RULE:

You MUST extract total_amount ONLY from numeric values (with currency if present) that appear in the <document> text.

You MUST NOT calculate anything.

You MUST NOT guess.

You MUST NOT copy or reuse any number from the examples in this prompt unless that exact number appears in the current <document> text.

How to detect:
Look for phrases such as:

"TOTAL", "TOTAL AMOUNT", "TOTAL AMOUNT DUE"

"TOTAL DUE", "AMOUNT DUE", "TOTAL THIS INVOICE"

"TOTAL INVOICE", "GRAND TOTAL"

"NET AMOUNT PAYABLE", "TOTAL PAYABLE"

"BALANCE DUE", "AMOUNT PAYABLE", "AMOUNT TO BE PAID"

"GROSS TOTAL", "GROSS AMOUNT", "GROSS WORTH", "GROSS INVOICE VALUE", "TOTAL INVOICE VALUE"

A "SUMMARY" table that includes columns like "Net worth", "VAT", "Gross worth", followed by a "Total" row.

Priority rules:

If the invoice has a field explicitly labeled with text like "TOTAL AMOUNT DUE", "AMOUNT DUE", "BALANCE DUE", "NET AMOUNT PAYABLE", "AMOUNT PAYABLE", "TOTAL PAYABLE", "TOTAL DUE", or "TOTAL THIS INVOICE", use that amount.

Otherwise, if there is a SUMMARY table that shows columns like "Net worth", "VAT", and "Gross worth", and the OCR produces a final "Total" row that may appear flattened, for example:
"Total $ 5 640,17 $ 564,02 $ 6 204,19"
then:

The FIRST number after "Total" is usually Net worth (pre-tax).

The SECOND number is VAT.

The LAST number is Gross worth (Net worth + VAT).
In this layout:

You MUST return the LAST number from that "Total" row (the Gross worth), because that is the full amount owed.

You MUST NOT return the first number after "Total". That first number is only the net amount before tax and is NOT the final amount payable.

If (1) and (2) do not apply, but there is a single commercial total like "Gross worth", "Gross Total", "Gross Invoice Value", "Total Invoice Value", and there is no clearer "Amount Due" or "Net Amount Payable", you may return that gross amount.

If multiple totals for different currencies or cost centers appear and it is not clear which applies to the billed party, set total_amount to null.

Rules:

Return ONLY the currency/amount string exactly as it appears, including currency symbols, commas, spaces, decimals, minus signs, and currency codes.
Examples:
"$12,450.00"
"$ 6 204,19"
"₹ 1,25,999.00"
"1.250,00 EUR"
"USD 750.00"
"-$42.10"

Do NOT reformat.

Do NOT insert or assume a currency symbol.

Do NOT compute or add values yourself.

Do NOT copy any amount from the examples unless that exact amount appears in the current <document> text.

Null handling:

If you cannot clearly identify the final payable amount using the above logic, return "total_amount": null.

MISSING / UNCERTAIN FIELDS

If a field cannot be confidently found or is ambiguous based on the rules above, you MUST return null for that field.

Example of null usage:
"invoice_date": null

This requirement overrides any attempt to guess. Null is always better than guessing.

EXAMPLES

The example outputs below are ONLY demonstrations of format.
You MUST NOT copy any names, addresses, dates, invoice numbers, tax IDs, company names, currency values, VAT values, totals, or any other content from the examples below into your final JSON unless those exact strings also appear in the <document> text you are currently analyzing.

Example A (simple header style)
Input:
<document>
INVOICE
DATE: March 8, 1984
TERMS: Net
INVOICE NO.: 31T 2552-14B

BILL TO:
Lorillard Research Center
Post Office Box 21688
Greensboro, N. C. 27420
ATTN: Dr. Harry Minnemeyer
Director, Research

TOTAL AMOUNT DUE: $12,450.00
</document>

Output:
{"invoice_number":"31T 2552-14B","invoice_date":"March 8, 1984","billing_address":"Lorillard Research Center\nPost Office Box 21688\nGreensboro, N. C. 27420\nDr. Harry Minnemeyer\nDirector, Research","total_amount":"$12,450.00"}

Example B (summary table style with Gross worth as final payable)
Input:
<document>
Invoice no: 51109338
Date of issue: 04/13/2013

Seller:
Andrews, Kirby and Valdez
58861 Gonzalez Prairie
Lake Daniellefurt, IN 57228

Client:
Becker Ltd
8012 Stewart Summit Apt. 455
North Douglas, AZ 95355

SUMMARY
VAT [%] Net worth VAT Gross worth
10% 5 640,17 564,02 6 204,19

Total $ 5 640,17 $ 564,02 $ 6 204,19
</document>

Output:
{"invoice_number":"51109338","invoice_date":"04/13/2013","billing_address":"Becker Ltd\n8012 Stewart Summit Apt. 455\nNorth Douglas, AZ 95355","total_amount":"$ 6 204,19"}

In this style:

"Client:" is used as the billed party, so we use that block for billing_address.

We do NOT include Tax Id, IBAN, phone numbers, etc.

The "Total" row in SUMMARY contains multiple numeric columns (Net worth, VAT, Gross worth). We MUST return the LAST number in that row, which corresponds to "Gross worth", not the first number. The last number is the final payable amount including tax.

FINAL INSTRUCTION (CRITICAL)

ABSOLUTE COMPLIANCE RULES:

You MUST output ONLY the final JSON object, minified, with fields in this exact order: invoice_number, invoice_date, billing_address, total_amount.

You MUST NOT include any explanation, reasoning, apology, analysis, debugging info, markdown, or commentary.

You MUST NOT invent or guess values.

You MUST NOT copy any sample values (names, addresses, numbers, currency amounts) from the examples in this prompt unless those exact values appear in the <document> text for the current invoice.

If any field cannot be confidently extracted from the <document> text alone, that field MUST be null.
"""
    )
    return f"{instructions}\n\n<document>\n{markdown_text}\n</document>"


def parse_response(response_text: str) -> Dict[str, Optional[str]]:
    LOGGER.debug("Raw model response: %s", response_text)
    match = re.search(r"\{.*\}", response_text, flags=re.DOTALL)
    if not match:
        raise ValueError("Model response does not contain JSON")

    candidate = match.group(0)
    payload = json.loads(candidate)

    for key in ("invoice_number", "billing_address", "total_amount"):
        payload.setdefault(key, None)

    return payload


def run_extraction(
    markdown_text: str,
    model,
    processor,
    max_new_tokens: int = 512,
) -> ExtractionResult:
    prompt = build_prompt(markdown_text)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                }
            ],
        },
        
    ]

    chat_text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    inputs = processor(text=[chat_text], padding=True, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.inference_mode():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
        )

    generated_ids = output_ids[:, inputs["input_ids"].shape[1] :]
    raw_text = processor.batch_decode(
        generated_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True,
    )[0]

    payload = parse_response(raw_text)
    return ExtractionResult.from_json(payload)


def iterate_markdown_files(source_dir: Path) -> Iterable[Path]:
    return sorted(source_dir.glob("*.md"))


def process_directory(
    source_dir: Path,
    output_dir: Path,
    model,
    processor,
    overwrite: bool = True,
) -> None:
    output_dir.mkdir(parents=True, exist_ok=True)

    for md_file in iterate_markdown_files(source_dir):
        target_path = output_dir / (md_file.stem + ".json")
        if target_path.exists() and not overwrite:
            LOGGER.info("Skipping %s (already processed)", md_file.name)
            continue

        LOGGER.info("Processing %s", md_file)
        markdown_text = md_file.read_text(encoding="utf-8", errors="ignore")

        try:
            result = run_extraction(
                markdown_text,
                model=model,
                processor=processor,
            )
        except Exception as exc:  # noqa: BLE001
            LOGGER.error("Extraction failed for %s: %s", md_file, exc)
            (output_dir / f"{md_file.stem}_error.txt").write_text(
                f"Error: {exc}\n", encoding="utf-8"
            )
            continue

        target_path.write_text(
            json.dumps(result.to_dict(), indent=2, ensure_ascii=False) + "\n",
            encoding="utf-8",
        )


def resolve_directories(root: Path) -> Dict[str, Dict[str, Path]]:
    return {
        "nanonet": {
            "source": root / "nanonet_markdown_output",
            "target": root / "nanonet_json_output",
        },
        "paddle": {
            "source": root / "paddle_markdown_output",
            "target": root / "paddle_json_output",
        },
        "tesseract": {
            "source": root / "tesseract_markdown_output",
            "target": root / "tesseract_json_output",
        },
    }


configure_logging(verbose=False)
project_root = Path.cwd()
directories = resolve_directories(project_root)

directories

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


{'nanonet': {'source': PosixPath('/home/ec2-user/SageMaker/notebook/nanonet_markdown_output'),
  'target': PosixPath('/home/ec2-user/SageMaker/notebook/nanonet_json_output')},
 'paddle': {'source': PosixPath('/home/ec2-user/SageMaker/notebook/paddle_markdown_output'),
  'target': PosixPath('/home/ec2-user/SageMaker/notebook/paddle_json_output')},
 'tesseract': {'source': PosixPath('/home/ec2-user/SageMaker/notebook/tesseract_markdown_output'),
  'target': PosixPath('/home/ec2-user/SageMaker/notebook/tesseract_json_output')}}

## 2. Load the Nanonets OCR Model
This step downloads the model weights (if needed) and initializes the tokenizer and processor. It may take a few minutes the first time. If FlashAttention 2 isn't installed, the helper automatically falls back to the standard attention implementation. Set `target_device` to `"cpu"`, `"cuda:0"`, or any other available device string before running the next cell.

In [3]:
target_device = "cuda:0" if torch.cuda.is_available() else "cpu"
model, tokenizer, processor = load_llm(device=target_device)
model.device

2025-10-27 19:16:56,578 [INFO] Loading model 'nanonets/Nanonets-OCR-s'
2025-10-27 19:16:56,579 [INFO] Target device: cuda:0
2025-10-27 19:16:56,579 [WARNING] FlashAttention 2 not available; falling back to default attention. Install flash-attn to enable it.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


device(type='cuda', index=0)

## 3. Extract Fields from a Single Markdown File
Pick any markdown file from the available directories and run `run_extraction` to obtain the JSON payload. Adjust the `sample_path` as needed.

In [4]:
sample_source = directories["nanonet"]["source"]
markdown_files = sorted(sample_source.glob("*.md"))
if not markdown_files:
    raise FileNotFoundError(f"No markdown files found in {sample_source}")

sample_path = markdown_files[0]
print(f"Using sample file: {sample_path}")

markdown_text = sample_path.read_text(encoding="utf-8", errors="ignore")
extraction = run_extraction(
    markdown_text,
    model=model,
    processor=processor,
    max_new_tokens=4000,
)
extraction.to_dict()

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Using sample file: /home/ec2-user/SageMaker/notebook/nanonet_markdown_output/0001139626.md


{'invoice_number': None, 'billing_address': None, 'total_amount': None}

## 3. Extract Fields from a Single Markdown File
Pick any markdown file from the available directories and run `run_extraction` to obtain the JSON payload. The cell prints the first lines of the markdown so you can confirm the input text before reviewing the extracted fields.

In [5]:
single_output_dir = directories["nanonet"]["target"]
single_output_dir.mkdir(parents=True, exist_ok=True)
output_path = single_output_dir / f"{sample_path.stem}.json"
output_path.write_text(
    json.dumps(extraction.to_dict(), indent=2, ensure_ascii=False) + "\n",
    encoding="utf-8",
)
print(f"Wrote JSON to {output_path}")

Wrote JSON to /home/ec2-user/SageMaker/notebook/nanonet_json_output/0001139626.json


## 5. Batch Process All OCR Outputs
Process all markdown files from all three OCR engines (Nanonet, Paddle, Tesseract) and save JSON results.

In [6]:
# Process all three OCR sources
for source_name in ["nanonet", "paddle", "tesseract"]:
    source_dir = directories[source_name]["source"]
    output_dir = directories[source_name]["target"]
    
    print(f"\n{'='*70}")
    print(f"Processing {source_name.upper()} markdown files")
    print(f"Source: {source_dir}")
    print(f"Target: {output_dir}")
    print(f"{'='*70}\n")
    
    # Count files
    md_files = list(source_dir.glob("*.md"))
    print(f"Found {len(md_files)} markdown files to process\n")
    
    if not md_files:
        print(f"⚠️  No markdown files found in {source_dir}, skipping...\n")
        continue
    
    # Process the directory
    process_directory(
        source_dir=source_dir,
        output_dir=output_dir,
        model=model,
        processor=processor,
        overwrite=True  # Set to False to skip already-processed files
    )
    
    # Summary
    json_files = list(output_dir.glob("*.json"))
    error_files = list(output_dir.glob("*_error.txt"))
    print(f"\n✅ Completed {source_name}:")
    print(f"   - Successfully processed: {len(json_files)} files")
    print(f"   - Errors: {len(error_files)} files")

print("\n" + "="*70)
print("🎉 Batch processing complete!")
print("="*70)

2025-10-27 19:17:05,709 [INFO] Processing /home/ec2-user/SageMaker/notebook/nanonet_markdown_output/0001139626.md
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Processing NANONET markdown files
Source: /home/ec2-user/SageMaker/notebook/nanonet_markdown_output
Target: /home/ec2-user/SageMaker/notebook/nanonet_json_output

Found 61 markdown files to process



2025-10-27 19:17:07,760 [INFO] Processing /home/ec2-user/SageMaker/notebook/nanonet_markdown_output/0001139716.md
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
2025-10-27 19:17:09,368 [INFO] Processing /home/ec2-user/SageMaker/notebook/nanonet_markdown_output/0013046347.md
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
2025-10-27 19:17:11,348 [INFO] Processing /home/ec2-user/SageMaker/notebook/nanonet_markdown_output/00920576.md
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
2025-10-27 19:17:15,293 [INFO] Processing /home/ec2-user/SageMaker/notebook/nanonet_markdown_output/00921466.md
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
2025-10-27 1


✅ Completed nanonet:
   - Successfully processed: 61 files
   - Errors: 0 files

Processing PADDLE markdown files
Source: /home/ec2-user/SageMaker/notebook/paddle_markdown_output
Target: /home/ec2-user/SageMaker/notebook/paddle_json_output

Found 61 markdown files to process



2025-10-27 19:20:39,619 [INFO] Processing /home/ec2-user/SageMaker/notebook/paddle_markdown_output/0001139716.md
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
2025-10-27 19:20:43,211 [INFO] Processing /home/ec2-user/SageMaker/notebook/paddle_markdown_output/0013046347.md
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
2025-10-27 19:20:45,170 [INFO] Processing /home/ec2-user/SageMaker/notebook/paddle_markdown_output/00920576.md
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
2025-10-27 19:20:48,990 [INFO] Processing /home/ec2-user/SageMaker/notebook/paddle_markdown_output/00921466.md
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
2025-10-27 19:20


✅ Completed paddle:
   - Successfully processed: 61 files
   - Errors: 0 files

Processing TESSERACT markdown files
Source: /home/ec2-user/SageMaker/notebook/tesseract_markdown_output
Target: /home/ec2-user/SageMaker/notebook/tesseract_json_output

Found 61 markdown files to process



2025-10-27 19:23:54,568 [INFO] Processing /home/ec2-user/SageMaker/notebook/tesseract_markdown_output/0001139716.md
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
2025-10-27 19:23:56,367 [INFO] Processing /home/ec2-user/SageMaker/notebook/tesseract_markdown_output/0013046347.md
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
2025-10-27 19:23:58,298 [INFO] Processing /home/ec2-user/SageMaker/notebook/tesseract_markdown_output/00920576.md
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
2025-10-27 19:24:01,729 [INFO] Processing /home/ec2-user/SageMaker/notebook/tesseract_markdown_output/00921466.md
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
2025


✅ Completed tesseract:
   - Successfully processed: 61 files
   - Errors: 0 files

🎉 Batch processing complete!


## 6. Test Single File Extraction (Optional)
Test extraction on a single file to preview results before batch processing.

In [7]:
source_name = "paddle"  # change to "paddle" or "tesseract" as needed
source_dir = directories[source_name]["source"]
markdown_files = sorted(source_dir.glob("*.md"))
if not markdown_files:
    raise FileNotFoundError(f"No markdown files found in {source_dir}")

sample_path = markdown_files[0]
print(f"From folder: {source_dir}")
print(f"Using sample file: {sample_path}")

markdown_text = sample_path.read_text(encoding="utf-8", errors="ignore")
print("\nFirst 20 lines of markdown content:\n")
print("\n".join(markdown_text.splitlines()[:20]))

extraction = run_extraction(
    markdown_text,
    model=model,
    processor=processor,
    max_new_tokens=512,
)
print("\nExtracted fields:")
extraction.to_dict()

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


From folder: /home/ec2-user/SageMaker/notebook/paddle_markdown_output
Using sample file: /home/ec2-user/SageMaker/notebook/paddle_markdown_output/0001139626.md

First 20 lines of markdown content:

## Page 1

<!-- Layout: two_column (2 column(s)) -->

```
RUN GN AUG09/74 Ar 19.59 PRODUCTIDM BILL iILL MUMEER P-O8-O933 PAGk DATE DVE AUG23/79 CLIENT BW GRUWN & WILLIAMSON TOBACCD CORP 1600 W HILL ST PRODUCT KM KOOL MILDS LOUISVILLk KY 40201 JDB PMS8I2 KOUL MILDS PACK PRESLNTATION GEDIA P ESTEMATE UMBZR= KM-MISC-78-2RI BEW CODE= N/A PRINT PROON
```

DESCAIPTION VENDOR NAME NET AMOUNT COMMISSION TO PROTUGRPHRS FEE TEST-COPY PROD|CTION 122178 PO NU 3935 THOM DE SANTO 1500.00 1500. 781011 PO kO 3760 SHIG IKEDA 1000.00 1000 010479 PO NO 4042 J. RUDOLPH YESA 1146.04 1146 K/C04~INCLUDES EXPENSES FOK 5396.04 112076 PD N0 3758 J. RUDOLPH TESA 2640.65 264 /C04-INCLUVES EXPENSES FOR RELEASED 1240.65 AUG 2 7 1979 4WONK-CODE TCTALS4 6286.69 62 huv. vEPT. YE TRANSFER 8914 AD NO 6452 AUTHENTICOLOR 1900.0

{'invoice_number': None, 'billing_address': None, 'total_amount': None}

## 7. Inspect Generated JSON Files
List the JSON outputs and preview one to confirm the extracted values.

In [8]:
# Inspect results from all OCR sources
for source_name in ["nanonet", "paddle", "tesseract"]:
    output_dir = directories[source_name]["target"]
    json_files = sorted(output_dir.glob("*.json"))
    error_files = sorted(output_dir.glob("*_error.txt"))
    
    print(f"\n{'='*60}")
    print(f"{source_name.upper()} Results:")
    print(f"{'='*60}")
    print(f"✓ Success: {len(json_files)} JSON files")
    print(f"✗ Errors: {len(error_files)} files")
    
    if json_files:
        # Preview first file
        preview_path = json_files[0]
        preview_data = json.loads(preview_path.read_text(encoding="utf-8"))
        print(f"\nSample extraction from {preview_path.name}:")
        for key, value in preview_data.items():
            print(f"  {key}: {value}")

print("\n" + "="*60)


NANONET Results:
✓ Success: 61 JSON files
✗ Errors: 0 files

Sample extraction from 0001139626.json:
  invoice_number: None
  billing_address: None
  total_amount: None

PADDLE Results:
✓ Success: 61 JSON files
✗ Errors: 0 files

Sample extraction from 0001139626.json:
  invoice_number: None
  billing_address: None
  total_amount: None

TESSERACT Results:
✓ Success: 61 JSON files
✗ Errors: 0 files

Sample extraction from 0001139626.json:
  invoice_number: None
  billing_address: None
  total_amount: None



## Next Steps
- Tune `max_new_tokens` or edit the `build_prompt` helper above if invoices require longer or differently formatted outputs.
- Compare the JSON results across OCR sources to evaluate quality differences.
- Integrate the JSON files into downstream analytics or storage systems.